## **Columns/Features**

**Country** Country

**Year** Year

**Status** Developed or Developing status

**Life expectancy** Life Expectancy in age

**Adult Mortality** Adult Mortality Rates of both sexes (probability of dying between 15 and 60 years per 1000 population)

**infant** deaths Number of Infant Deaths per 1000 population

**Alcohol** Alcohol, recorded per capita (15+) consumption (in litres of pure alcohol)
percentage 

**expenditure** Expenditure on health as a percentage of Gross Domestic Product per capita(%)

**Hepatitis B** Hepatitis B (HepB) immunization coverage among 1-year-olds (%)

**Measles** Measles - number of reported cases per 1000 population

**BMI** Average Body Mass Index of entire population

**under-five deaths** Number of under-five deaths per 1000 population

**Polio** Polio (Pol3) immunization coverage among 1-year-olds (%)

**Total expenditure** General government expenditure on health as a percentage of total government expenditure (%)

**Diphtheria** Diphtheria tetanus toxoid and pertussis (DTP3) immunization coverage among 1-year-olds (%)

**HIV/AIDS** Deaths per 1 000 live births HIV/AIDS (0-4 years)

**GDP** Gross Domestic Product per capita (in USD)

**Population** Population of the country

**thinness 1-19 years** Prevalence of thinness among children and adolescents for Age 10 to 19 (% )

**thinness 5-9 years** Prevalence of thinness among children for Age 5 to 9(%)

**Income composition of resources** Human Development Index in terms of income composition of resources (index ranging from 0 to 1)

**Schooling** Number of years of Schooling(years) 

In [1]:

%%capture
!pip install eli5
!pip install pdpbox
!pip install shap
!pip install category_encoders==2.*
!pip install pandas-profiling==2.*

In [2]:
# Importing Libraries needed for the project. 
import pandas_profiling
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_absolute_error, r2_score
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeRegressor
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import eli5
from eli5.sklearn import PermutationImportance
%matplotlib inline
plt.rcParams['figure.dpi'] = 72
from pdpbox.pdp import pdp_isolate, pdp_plot
from pdpbox.pdp import pdp_interact, pdp_interact_plot
import seaborn as sns
import plotly.graph_objects as go
import shap;

KeyboardInterrupt: 

## **Data Leakage Features and Why**

**Income composition of resources** - The Human Development Index (HDI) is a statistic composite index of life expectancy, education, and per capita income indicators, which are used to rank countries into four tiers of human development. A country scores a higher HDI when the lifespan is higher, the education level is higher, and the gross national income GNI (PPP) per capita is higher. https://en.wikipedia.org/wiki/Human_Development_Index


**expenditure** is using GDP and total expenditure

**infant mortality** could also be representated by under_five_deaths

**under_five_deaths and adult mortality** Adult mortality is “The number of people that die between ages 15 and 60 yrs old per 1000 people” If this feature is in the predictive model, then the insight would be “In countries where more people die young, people don’t live as long on average.”

In [0]:
# Loading the data
data = 'https://raw.githubusercontent.com/EvidenceN/Life-Expectancy-Prediction/master/Life%20Expectancy/Data/Life%20Expectancy%20Data.csv'

life = pd.read_csv(data)

print(life.shape)
life.head()

In [0]:
# Get a profile report of the dataset. 
profile = pandas_profiling.ProfileReport(life)
profile

**Dealing with missing data.** A lot of GDP and Population data is missing for some countries. 

Due to the nature of the dataset, i can not use imputation with a mean or median strategy. If i use imputation, then i am taking information from other countries and putting it in for a different country. 

for example, i will be taking population and GDP information from a different countries and imputing it as the population and GDP of a different countries. That information would be inaccurate. 

I could meticoulously look up the information from the countries with missing data, but that would consume a lot of time. 

Drop rows with missing gdp and population information. That would leave me with 145 countries instead of 193 and 
75% of the total data. By doing this, i will loose 25% of the data. 

for other features like Bmi, thinness, hepatitis B with missing values, i will input 0's for those missing values. 
if values are missing for those features, 0 is a safe assumption and won't skew the data or disrupt the interpretation
of the data

XGboost can handle nan, but other models might struggle to handle nan, since imputation is not a viable option, then
i will use 0 in place of nan's

drop country and years because we are not trying to see if the life exptancy of a country changes from year to year. The country and year the data originated from should 
not be a predictor of life expectancy. 

The objective is to use health and economic factors to predict life expectancy. Having data from different countries over the
years allows for diversity of information. 

In [0]:
# drop gdp and population and life expectancy missing values
life = life.dropna(subset=['GDP', 'Population', 'Life_expectancy_'])

In [0]:
# get the mean baseline because this is a regression problem
# with regression, the baseline can be as simple as the mean.

mean_baseline = life['Life_expectancy_'].mean()
errors = mean_baseline - life['Life_expectancy_']
mean_absolute_error_ = errors.abs().mean()
print(f'Mean Baseline: {mean_baseline:.1f} years')
print('Without any other factors, life expectancy could be guessed to be')
print(f'{mean_baseline:.1f}yrs and will be off by a mean absolute error of {round(mean_absolute_error_, 2)}')

In [0]:
# split into train, test, validation 15% for testing, 15% for validation, 70% training. 
# No stratification in splitting because of ValueError: The least populated class in y has 
# only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

from sklearn.model_selection import train_test_split

train, test = train_test_split(life, test_size=0.15, random_state=42)

train, val = train_test_split(train, test_size=0.20, random_state=42)

In [0]:
# function to perform data cleaning on all datasets. 

def wrangle(x):
    x = x.copy()
    
    x.columns = [col.lower()         # make column names lowercase
                .strip('_')         # strip leading/trailing underscores
                .replace('_', ' ')  # replace remaining punctuation with spaces
                .replace('-', ' ') 
                .replace('/', ' ')
                for col in x.columns]
    
    # dropping columns with leakage
    x = x.drop(columns = ['country', 'year', 'income composition of resources',
                         'percentage expenditure', 'infant deaths', 
                          'under five deaths', 'adult mortality'])
    
    # filling nan values with 0 because I can't do imputation and the missing
    # values can be assumed to be 0.
    
    x = x.fillna(value=0)
    
    return x

In [0]:
# clean all data the same way
train = wrangle(train)
test = wrangle(test)
val = wrangle(val)

In [0]:
# defining x and y values for training, validation, and test sets. 

target = 'life expectancy'
features = train.columns.drop(target)

x_train = train[features]
y_train = train[target]
x_val = val[features]
y_val = val[target]
x_test = test[features]
y_test = test[target]

In [0]:
# Function to build multiple Models at once. 

def model(x_train, y_train, x_val, y_val):

  # Defining Global Coefficients so that it can be used
  # outside of the function. 
  global coefficients_lr, coefficients_ridge, importances_tree
  global importances_boost, importances_forest, forest

  ##lr = linear regression model
  
  lr = make_pipeline(
      ce.OneHotEncoder(),
      LinearRegression())
  
  # Getting error and R^2 scores from linear regression model. 
  # fitting the model from linear regression pipeline
  lr.fit(x_train, y_train)
  # R2 score from linear regression model
  lr_score = lr.score(x_val, y_val)
  print(f'LinearRegression R^2 Score {lr_score}')
  # mean absolute error from linear regression model
  y_pred_lr = lr.predict(x_val)
  mae_lr = mean_absolute_error(y_val, y_pred_lr)
  print(f'LinearRegression mean_absolute_error {mae_lr}\n' )
  # coefficients from linear regression model
  coeff_lr = lr.named_steps['linearregression'].coef_
  print(f'LinearRegression Coefficient {coeff_lr}')
  #Intercept from linear regression model. 
  intercept_lr = lr.named_steps['linearregression'].intercept_
  print(f'LinearRegression Intercept {intercept_lr}\n')

  # plotting the coefficients from linear regression model.
  encoder = lr.named_steps['onehotencoder']
  columns_lr = encoder.transform(x_val).columns
  coefficients_lr = pd.Series(coeff_lr, columns_lr)

  ## Ridge Regression Model

  ridge = make_pipeline(
        ce.OneHotEncoder(),
        StandardScaler(),
        Ridge(alpha=10))

  # fitting the model
  ridge.fit(x_train, y_train)

  # mean absolute error for ridge regresssion model
  y_pred_ridge = ridge.predict(x_val)
  mae_ridge = mean_absolute_error(y_val, y_pred_ridge)
  print(f'Ridge regression mean_absolute_error {mae_ridge}' )

  # R2 score for ridge regresssion model
  ridge_score = ridge.score(x_val, y_val,)
  print(f'Ridge Regression Score R^2 Score {ridge_score}\n')

  coeff_ridge = ridge.named_steps['ridge'].coef_
  print(f'Ridge Regression Coefficient {coeff_ridge}')

  intercept_ridge = ridge.named_steps['ridge'].intercept_
  print(f'Ridge Regression Intercept {intercept_ridge}\n')

  # plotting the coefficients for ridge regresssion model
  encoder = ridge.named_steps['onehotencoder']
  columns_ridge = encoder.transform(x_val).columns
  coefficients_ridge = pd.Series(coeff_ridge, columns_ridge)

  ## Decision Tree Model

  tree = make_pipeline(
    ce.OrdinalEncoder(),
    DecisionTreeRegressor(random_state=42))

  tree.fit(x_train, y_train)

  # mean absolute error for decision tree model
  y_pred = tree.predict(x_val)
  mae = mean_absolute_error(y_val, y_pred)
  print(f'Decision tree mean_absolute_error {mae}' )

  # R2 score for decision tree model
  tree_score = tree.score(x_val, y_val,)
  print(f'Decision tree Score R^2 Score {tree_score}\n')

  # plotting the feature importances for decision tree model
  tree_model = tree.named_steps['decisiontreeregressor']
  encoder = tree.named_steps['ordinalencoder']
  columns = encoder.transform(x_val).columns
  importances_tree = pd.Series(tree_model.feature_importances_, columns)

  ## random forest model

  forest = make_pipeline(
      ce.OrdinalEncoder(),
      RandomForestRegressor(random_state=42, n_estimators=100, n_jobs=-1)
  )

  forest.fit(x_train, y_train)

  # mean absolute error for random forest model
  y_pred_val = forest.predict(x_val)
  mae = mean_absolute_error(y_val, y_pred_val)
  print(f'random forest mean_absolute_error {mae}' )

  # R2 score for random forest model
  forest_score = forest.score(x_val, y_val,)

  print(f'random forest Score R^2 Score {forest_score}\n')

  # plotting the feature importances for random forest model
  model_forest = forest.named_steps['randomforestregressor']
  encoder = forest.named_steps['ordinalencoder']
  columns = encoder.transform(x_val).columns
  importances_forest = pd.Series(model_forest.feature_importances_, columns)

  ## gradient boosting model

  boost = make_pipeline(
      ce.OrdinalEncoder(),
      XGBRegressor(random_state=42, n_estimators=100, n_jobs=-1)
  );

  boost.fit(x_train, y_train)

  # mean absolute error for gradient boosting model
  y_pred_boost = boost.predict(x_val)
  mae_boost = mean_absolute_error(y_val, y_pred_boost)
  print(f'gradient boosting mean_absolute_error {mae_boost}' )

  # R2 score for gradient boosting model
  boost_score = boost.score(x_val, y_val,)
  print(f'gradient boosting R^2 Score {boost_score}')

  # defining the dataframe for plotting the feature importances for gradient boosting model
  model_boost = boost.named_steps['xgbregressor']
  encoder = boost.named_steps['ordinalencoder']
  columns = encoder.transform(x_val).columns
  importances_boost = pd.Series(model_boost.feature_importances_, columns);

In [0]:
model(x_train, y_train, x_val, y_val)

In [0]:
# Visualization for Linear Regression model.
coefficients_lr.sort_values().plot.barh(color='grey')
plt.title('Visualization for Linear Regression Model Coefficients')

In [0]:
# Visualization for Ridge Regression model.
coefficients_ridge.sort_values().plot.barh(color='grey')
plt.title('Visualization for Ridge Regression Model Coefficients')

In [0]:
# Visualization for Decision model.
importances_tree.sort_values().plot.barh(color='grey')
plt.title('Visualization for Decision Tree Model Feature Importances')

In [0]:
# Visualization for Random Forest model.
importances_forest.sort_values().plot.barh(color='grey')
plt.title('Visualization for Random Forest Model Feature Importances')

In [0]:
# Visualization for gradient boost model.
importances_boost.sort_values().plot.barh(color='grey')
plt.title('Visualization for Gradient Boosting Model Feature Importances')

In [0]:
# using the best model which is the random forest model for prediction
# with the test data set. 
y_pred_test = forest.predict(x_test)
mae = mean_absolute_error(y_test, y_pred_test)
print(f'random forest test mean_absolute_error {mae}' )

# R2 score for test data set
forest_score = forest.score(x_test, y_test,)
print(f'random forest test Score R^2 Score {forest_score}')

In [0]:
# Permutation Importance Function. Eli5 - the library used for permutation
# importance calculations doesn't work with pipeline models. So, everything has
# to be preprocess before putting it in a model. 

def permutation(x_train, y_train, x_val, y_val):

  # Pre processing the data using a pipeline
  pre_processing = make_pipeline(
      ce.OrdinalEncoder())

  x_train_encoded =  pre_processing.fit_transform(x_train)
  x_val_encoded =  pre_processing.transform(x_val)

  # fitting the model
  model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
  model.fit(x_train_encoded, y_train)

  # instantiating and setting the permutation parameters
  permuter = PermutationImportance(
      model,
      scoring = 'r2', 
      n_iter = 5,
      random_state = 42
  )

  # fitting the permutation importance to the validation dataset
  permuter.fit(x_val_encoded, y_val)

  # getting and plotting the graph from the permution calculations
  feature_names = x_val.columns.to_list()
  pd.Series(permuter.feature_importances_, feature_names).sort_values(ascending=False)

  metric = eli5.show_weights(
      permuter,
      top=None,
      feature_names=feature_names,
  )

  return metric

In [0]:
permutation(x_train, y_train, x_val, y_val)

In [0]:
# Function for single feature partial dependence plot(PDP)

def pdp(feature, title="", ylabel="", xlabel="", xlim=(0,10)):

  # defining the parameters for the pdp plot
  isolated = pdp_isolate(
    model=forest,
    dataset=x_val,
    model_features = x_val.columns,
    feature = feature,
    num_grid_points=50
    )

  # plotting the isolated features
  pdp_plot(isolated, feature_name=feature)
  plt.xlim(xlim)
  plt.title(title)
  plt.ylabel(ylabel)
  plt.xlabel(xlabel)

In [0]:
# pdp plot for HIV AIDS feature
pdp('hiv aids', 
    title = "Partial Dependence of life expectancy on 'HIV AIDS'",
    ylabel='Life Expectancy',
    xlabel='Death by HIV Aids in 0-4yr old per 1000 people')

In [0]:
# pdp plot for schooling feature
pdp('schooling', 
    title="Partial Dependence of life expectancy on 'Number of years in school'",
    ylabel='Life Expectancy',
    xlabel = 'Number of years in school',
    xlim=(None))

In [0]:
# pdp plot for schooling feature
pdp('bmi', 
    title="Partial Dependence of life expectancy on 'Body Mass Index'",
    ylabel='Life Expectancy',
    xlabel = 'Average Body Mass Index of entire population',
    xlim=(None))

In [0]:
#2d PDP Plot function

def pdp_2d(feature1, feature2, x_val, model, title=''):

  #Defining the parameters for the pdp interaction

  features=[feature1, feature2]

  interaction = pdp_interact(
    model=model,
    dataset=x_val,
    model_features = x_val.columns,
    features=features)
  
  # putting the pdp in a table format and plotting it

  # 2 options. 

  # option 1: This will just plot the pdp interaction

  #pdp_interact_plot(interaction, 
                    #plot_type='grid', feature_names=features)

  # Option 2: Taking the pdp interaction and plotting it using
  # a seaborn heatmap. 

  pdp = interaction.pdp.pivot_table(
    values='preds',
    columns=features[0],
    index=features[1])[::-1]

  plt.figure(figsize=(10,8))
  sns.heatmap(pdp, annot=True, fmt='.2f', cmap='viridis', xticklabels=pdp.columns.values.round(2),
                 yticklabels=pdp.index.values.round(2))
  plt.title(title);

In [0]:
# 2d pdp plot of HIV Aids and Schooling

pdp_2d('hiv aids', 'schooling', x_val, forest,
       title='Partial dependence of life expectancy on HIV AIDS death rate per 1000 people and number of years in school')


In [0]:
pdp_2d('hiv aids', 'bmi', x_val, forest,
       title='Partial dependence of life expectancy on HIV AIDS death rate per 1000 people and Body Mass Index')

In [0]:
pdp_2d('schooling', 'bmi', x_val, forest,
       title='Partial dependence of life expectancy on number of years in school and Body Mass Index')

In [0]:
# 3d partial independence plots function

def pdp_3d(feature1, feature2, x_val, model, target, title=''):

  #Defining the parameters for the pdp interaction

  features=[feature1, feature2]

  interaction = pdp_interact(
    model=model,
    dataset=x_val,
    model_features = x_val.columns,
    features=features)
  
  # putting the pdp in a table format and plotting it

  pdp = interaction.pdp.pivot_table(
    values='preds',
    columns=features[0],
    index=features[1])[::-1]

  # defining the graph area for the 3d plot

  surface = go.Surface(
      x=pdp.columns,
      y=pdp.index,
      z=pdp.values
  )

  # defining the layout of the 3d graph. 
  layout = go.Layout(
      scene=dict(
          xaxis=dict(title=features[0]), 
          yaxis=dict(title=features[1]), 
          zaxis=dict(title=target),),
          title=(title))

  #Plotting the 3d graph
  
  fig = go.Figure(surface, layout)
  fig.show()


In [0]:
pdp_3d('hiv aids', 'schooling', x_val, forest, 'life expectancy',
       title='Partial dependence of life expectancy on HIV AIDS death rate per 1000 people and number of years in school')

In [0]:
pdp_3d('hiv aids', 'bmi', x_val, forest, 'life expectancy', 
       title='Partial dependence of life expectancy on HIV AIDS death rate per 1000 people and Body Mass Index')

In [0]:
pdp_3d('schooling', 'bmi', x_val, forest, 'life expectancy', 
       title='Partial dependence of life expectancy on number of years in school and Body Mass Index')

In [0]:
# function to do shapley plots. 

def shapley(x_train, y_train, x_val, row_number=0):

  # encoding the x features from training and validation dataset. 

  encoding = ce.OrdinalEncoder()

  x_train_encoded = encoding.fit_transform(x_train)
  x_val_encoded = encoding.transform(x_val)

  model = RandomForestRegressor(n_estimators=100, n_jobs=-1, random_state=42)
  model.fit(x_train_encoded, y_train)

  # defining what row to examine in the shapley plot
  row = x_val_encoded.iloc[[row_number]]
  
  # predicting
  pred = model.predict(row)

  explainer = shap.TreeExplainer(model)
  shap_values = explainer.shap_values(row)

  feature_names = row.columns
  feature_values = row.values[0]
  shaps = pd.Series(shap_values[0], zip(feature_names, feature_values))
  
  # printing the result

  result = f'{pred[0]:.1f}yrs is the predicted life expectancy. \n\n'
  result += f'Starting from baseline of {explainer.expected_value:.1f}yrs \n\n'
  result += shaps.to_string()
  print(result)

  shap.initjs()

  return shap.force_plot(
      base_value = explainer.expected_value,
      shap_values = shap_values,
      features=row)



In [0]:
shapley(x_train, y_train, x_val)

In [0]:
shapley(x_train, y_train, x_val, 13)

In [0]:
shapley(x_train, y_train, x_val, 1)

In [0]:
shapley(x_train, y_train, x_val, 37)

In [0]:
import plotly.express as px

px.scatter(train, x='hiv aids', y='life expectancy', 
           title='Relationship between hiv aids and life expectancy')

In [0]:
px.scatter(train, x='schooling', y='life expectancy', 
           title='Relationship between schooling and life expectancy')

In [0]:
px.scatter(train, x='bmi', y='life expectancy', 
           title='Relationship between bmi and life expectancy')

In [0]:
px.scatter_3d(test, x='hiv aids', y='life expectancy', z='schooling',
           title='Relationship between hiv aids, schooling, and life expectancy')

In [0]:
px.scatter_3d(test, x='hiv aids', y='life expectancy', z='bmi',
           title='Relationship between hiv aids, BMI, and life expectancy')

In [0]:
px.scatter_3d(test, x='schooling', y='life expectancy', z='bmi',
           title='Relationship between BMI, schooling, and life expectancy')

In [0]:
# Feature Engineering

life.head()

Immunization Feature:
Engineer a new feature that looks combines child hood/infancy immunization statististics. 

From the current dataset, infancy immunization rate features are

Hepatitis B, Polio, Diphtheria. 

These features represent the percent of people that got immunization for those diseases between the ages of 0-1 yrs old. 

The new feature will combine all three immunization data to create a feature that tells how many infants got any form of the three immunizations. 

Hepatitis B, Polio, Diphtheria are in percentages. So the new feature will be calculated like this:

Get the z score of Hepatitis B, Polio, Diphtheria, then take the average of all three scores as my new feature. 

In [0]:
# function to calculate z score for all three immunization 
# features and compute the average z score. 

def z_score(x):
  x = x.copy()

  # calculate the z scores for the features of interest
  x['polio_z_score'] = (x['polio']-x['polio'].mean())/x['polio'].std()

  x['hepatitis_z_score'] = (x['hepatitis b']-x['hepatitis b'].mean())/x['hepatitis b'].std()

  x['diphtheria_z_score'] = (x['diphtheria']-x['diphtheria'].mean())/x['diphtheria'].std()

  # create a new feature that is the average of the z scores from all immunization features
  x['immunization'] = (x['polio_z_score'] + x['hepatitis_z_score'] 
                       + x['diphtheria_z_score'])/3

  # now drop the columns containing the original immunization features and the 
  # columns that has the individual z scores since there is a new single 
  # column(immunization) containing information from all 3 columns

  return x

In [0]:
# create a copy of the x_train, val, and test data sets above to avoid overiding them. 
x_train_new = x_train
x_val_new = x_val
x_test_new = x_test

In [0]:
x_train.head()

In [0]:
# apply feature engineering function to the new x test sets.
x_train_new = z_score(x_train_new)
x_val_new = z_score(x_val_new)
x_test_new = z_score(x_test_new)

In [0]:
# building the model using the new features. 

model(x_train_new, y_train, x_val_new, y_val)

In [0]:
# linear regression visualization for the new model. 
coefficients_lr.sort_values().plot.barh(color='grey')

In [0]:
# Random Forest visualization for the new model. 
importances_forest.sort_values().plot.barh(color='grey')

The new features doesn't make any difference in the model, So no further analysis will be done with it. 